In [ ]:
pip install catboost xgboost scikit-learn


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import roc_auc_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier


 Шаг 1. Разделение на обучающую и тестовую выборки (70/30)

In [ ]:
df = pd.read_csv('C:/Users/Public/Documents/data.csv')

X = df.drop(columns=['isFraud'])
y = df['isFraud']

# Разделим с random_state для воспроизводимости
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.3,
    random_state=42,
    stratify=y
)


Шаг 2. Обучение CatBoost "из коробки" и ROC AUC

In [ ]:
# 1. Категориальные признаки
cat_features = X.select_dtypes(include=['object']).columns.tolist()

# 2. Обработка пропусков
# Категориальные пропуски — 'missing'
X[cat_features] = X[cat_features].fillna('missing')
# Числовые пропуски — -999 (можно и median, если хочешь)
X = X.fillna(-999)

# 3. Кодируем категориальные признаки
encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X[cat_features] = encoder.fit_transform(X[cat_features])

# 4. Разделение на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# 5. Обучение CatBoost
model_cat = CatBoostClassifier(verbose=0, random_state=42)
model_cat.fit(X_train, y_train)
y_pred_cat = model_cat.predict_proba(X_test)[:, 1]
roc_auc_cat = roc_auc_score(y_test, y_pred_cat)
print(f"CatBoost ROC AUC: {roc_auc_cat:.4f}")

# 6. Предсказания и ROC AUC
y_pred_xgb = model_xgb.predict_proba(X_test)[:, 1]
roc_auc_xgb = roc_auc_score(y_test, y_pred_xgb)

print(f"XGBoost ROC AUC: {roc_auc_xgb:.4f}")


CatBoost ROC AUC: 0.9213


c:\Users\фвьшт\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:248: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()


XGBoost ROC AUC: 0.8911


 Шаг 3. Обучение XGBoost "из коробки" и ROC AUC

In [9]:
model_xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model_xgb.fit(X_train, y_train)

y_pred_xgb = model_xgb.predict_proba(X_test)[:, 1]

roc_auc_xgb = roc_auc_score(y_test, y_pred_xgb)
print(f"XGBoost ROC AUC: {roc_auc_xgb:.4f}")


c:\Users\фвьшт\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:248: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()
c:\Users\фвьшт\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:248: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()


XGBoost ROC AUC: 0.8911


 Сравнение моделей

In [10]:
if roc_auc_xgb > roc_auc_cat:
    print("XGBoost показал себя лучше в базовом варианте.")
else:
    print("CatBoost показал себя лучше в базовом варианте.")



CatBoost показал себя лучше в базовом варианте.


Шаг 4. Настройка XGBoost вручную для улучшения ROC AUC

In [11]:
model_xgb_tuned = XGBClassifier(
    n_estimators=300,
    learning_rate=0.05,
    max_depth=4,
    reg_alpha=0.5,
    reg_lambda=1,
    subsample=0.8,
    colsample_bytree=0.8,
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)

model_xgb_tuned.fit(X_train, y_train)
y_pred_xgb_tuned = model_xgb_tuned.predict_proba(X_test)[:, 1]
roc_auc_xgb_tuned = roc_auc_score(y_test, y_pred_xgb_tuned)
print(f"XGBoost (tuned) ROC AUC: {roc_auc_xgb_tuned:.4f}")


c:\Users\фвьшт\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:248: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()
c:\Users\фвьшт\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:248: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  feature_names = data.columns.format()


XGBoost (tuned) ROC AUC: 0.9102


 Финальный вывод

In [12]:
print(f"CatBoost ROC AUC: {roc_auc_cat:.4f}")
print(f"XGBoost (default) ROC AUC: {roc_auc_xgb:.4f}")
print(f"XGBoost (tuned) ROC AUC: {roc_auc_xgb_tuned:.4f}")

if roc_auc_xgb_tuned > roc_auc_cat:
    print("После настройки XGBoost стал лучше CatBoost.")
else:
    print("Даже после настройки XGBoost не обошёл CatBoost.")


CatBoost ROC AUC: 0.9213
XGBoost (default) ROC AUC: 0.8911
XGBoost (tuned) ROC AUC: 0.9102
Даже после настройки XGBoost не обошёл CatBoost.
